In [1]:
# Update to transformers 2.8.0
!pip install -q transformers --upgrade
!pip show transformers

Name: transformers
Version: 2.8.0
Summary: State-of-the-art Natural Language Processing for TensorFlow 2.0 and PyTorch
Home-page: https://github.com/huggingface/transformers
Author: Thomas Wolf, Lysandre Debut, Victor Sanh, Julien Chaumond, Sam Shleifer, Google AI Language Team Authors, Open AI team Authors, Facebook AI Authors, Carnegie Mellon University Authors
Author-email: thomas@huggingface.co
License: Apache
Location: /opt/conda/lib/python3.6/site-packages
Requires: tqdm, requests, boto3, regex, sentencepiece, numpy, sacremoses, tokenizers, filelock, dataclasses
Required-by: 


In [2]:
import os
import pickle

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, average_precision_score, roc_auc_score
import matplotlib.pyplot as plt
import transformers
from transformers import AutoTokenizer, TFAutoModel, TFElectraModel, ElectraTokenizer
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer

## Helper functions

In [3]:
def touch_dir(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)
        print(f"Created directory {dirname}.")
    else:
        print(f"Directory {dirname} already exists.")

In [4]:
def encode_qa(questions, answers, tokenizer, chunk_size=256, maxlen=512):
    """
    https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
    """
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
    
    for i in tqdm(range(0, len(questions), chunk_size)):
        q_chunk = questions[i:i+chunk_size].tolist()
        a_chunk = answers[i:i+chunk_size].tolist()
        text_chunk = list(zip(q_chunk, a_chunk))
        
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [5]:
def build_model(transformer, max_len=256):
    """
    https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
    """
    input_ids = L.Input(shape=(max_len, ), dtype=tf.int32)
    
    x = transformer(input_ids)[0]
    x = x[:, 0, :]
    x = L.Dense(1, activation='sigmoid', name='sigmoid')(x)
    
    # BUILD AND COMPILE MODEL
    model = Model(inputs=input_ids, outputs=x)
    model.compile(
        loss='binary_crossentropy', 
        metrics=['accuracy'], 
        optimizer=Adam(lr=1e-5)
    )
    
    return model

In [6]:
def save_model(model, transformer_dir='transformer'):
    """
    Special function to load a keras model that uses a transformer layer
    """
    transformer = model.layers[1]
    touch_dir(transformer_dir)
    transformer.save_pretrained(transformer_dir)
    sigmoid = model.get_layer('sigmoid').get_weights()
    pickle.dump(sigmoid, open('sigmoid.pickle', 'wb'))

In [7]:
def load_model(transformer_dir='transformer', max_len=256):
    """
    Special function to load a keras model that uses a transformer layer
    """
    transformer = TFElectraModel.from_pretrained(transformer_dir)
    model = build_model(transformer, max_len=max_len)
    sigmoid = pickle.load(open('sigmoid.pickle', 'rb'))
    model.get_layer('sigmoid').set_weights(sigmoid)
    
    return model

## TPU Configs

In [8]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


In [9]:
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
EPOCHS = 5
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192
MODEL = 'google/electra-base-discriminator'

## Load data

In [10]:
df = pd.read_csv('/kaggle/input/healthtap-eda/healthtap_qa_pairs.csv')

In [11]:
wrong1 = (
    df[['qid', 'answer']]
    .drop_duplicates("qid", keep="first").copy()
    .rename(columns={'qid': 'wrong_qid_1', 'answer': 'wrong_answer_1'})
)

wrong2 = (
    df[['qid', 'answer']]
    .drop_duplicates("qid", keep="last").copy()
    .rename(columns={'qid': 'wrong_qid_2', 'answer': 'wrong_answer_2'})
)

In [12]:
df = df.merge(wrong1, on='wrong_qid_1', how='left').merge(wrong2, on='wrong_qid_2', how='left')

# Remove this when actually running the real model
df = df.sample(n=500000, random_state=0)

## Bert tokenizer

In [13]:
# First load the real tokenizer
tokenizer = ElectraTokenizer.from_pretrained(MODEL)
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=True)
fast_tokenizer

Tokenizer(vocabulary_size=30522, model=BertWordPiece, add_special_tokens=True, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], clean_text=True, handle_chinese_chars=True, strip_accents=True, lowercase=True, wordpieces_prefix=##)

## Convert text to matrices

In [14]:
correct_ids = encode_qa(df.question, df.answer, fast_tokenizer, maxlen=MAX_LEN)
wrong_ids = encode_qa(df.question, df.wrong_answer_1, fast_tokenizer, maxlen=MAX_LEN)

In [15]:
input_ids = np.concatenate([correct_ids, wrong_ids])

labels = np.concatenate([
    np.ones(correct_ids.shape[0]),
    np.zeros(wrong_ids.shape[0])
]).astype(np.int32)

## Train test split

In [16]:
train_idx, test_idx = train_test_split(
    np.arange(input_ids.shape[0]), 
    test_size=0.3, 
    random_state=0
)

valid_idx, test_idx = train_test_split(
    test_idx, 
    test_size=0.5, 
    random_state=1
)

In [17]:
train_ids = input_ids[train_idx]
valid_ids = input_ids[valid_idx]
test_ids = input_ids[test_idx]

train_labels = labels[train_idx]
valid_labels = labels[valid_idx]
test_labels = labels[test_idx]

## Build datasets objects

In [18]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_ids, train_labels))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((valid_ids, valid_labels))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(test_ids)
    .batch(BATCH_SIZE)
)

## Modeling

In [19]:
%%time
with strategy.scope():
    transformer_layer = TFElectraModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 192)]             0         
_________________________________________________________________
tf_electra_model (TFElectraM ((None, 192, 768),)       108891648 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
sigmoid (Dense)              (None, 1)                 769       
Total params: 108,892,417
Trainable params: 108,892,417
Non-trainable params: 0
_________________________________________________________________
CPU times: user 22.8 s, sys: 6.39 s, total: 29.2 s
Wall time: 38.5 s


### Train model

In [20]:
n_steps = train_labels.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

Train for 5468 steps, validate for 1172 steps
Epoch 1/5
5468/5468 [==============================] - 1118s 204ms/step - loss: 0.3836 - accuracy: 0.8185 - val_loss: 0.3323 - val_accuracy: 0.8525
Epoch 2/5
5468/5468 [==============================] - 1018s 186ms/step - loss: 0.2948 - accuracy: 0.8675 - val_loss: 0.3041 - val_accuracy: 0.8709
Epoch 3/5
2710/5468 [=============>................] - ETA: 8:03 - loss: 0.1903 - accuracy: 0.9189

In [21]:
save_model(model)

Created directory transformer.


In [22]:
hist_df = pd.DataFrame(train_history.history)
hist_df.to_csv('train_history.csv')
hist_df

,loss,accuracy,val_loss,val_accuracy
0,0.383612,0.818492,0.332306,0.852513
1,0.294784,0.867532,0.304077,0.870933
2,0.251562,0.889388,0.298933,0.874887
3,0.215717,0.907003,0.322809,0.875213
4,0.184458,0.921615,0.338637,0.874813


## Eval

In [23]:
with strategy.scope():
    model = load_model(max_len=MAX_LEN)

In [24]:
y_score = model.predict(test_dataset, verbose=1).squeeze()
y_pred = y_score.round().astype(int)
print("AP:", average_precision_score(test_labels, y_score))
print("ROC AUC:", roc_auc_score(test_labels, y_score))
print(classification_report(test_labels, y_pred))

1172/1172 [==============================] - 104s 89ms/step
AP: 0.9477080538589123
ROC AUC: 0.9492718902254821
              precision    recall  f1-score   support

           0       0.88      0.86      0.87     75248
           1       0.86      0.88      0.87     74752

    accuracy                           0.87    150000
   macro avg       0.87      0.87      0.87    150000
weighted avg       0.87      0.87      0.87    150000

